***IMPORT DES DONNEES DE TRAFIC, BRUIT, METEO ET SONDAGE***

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append(r'C:\Users\martin.schoreisz\git\otv\otv\Transfert_Donnees')
sys.path.append(r'C:\Users\\martin.schoreisz\git\Outils\Outils\Martin_Perso')
sys.path.append(r'C:\Users\martin.schoreisz\git\PPBE_BdxMet\src\Carac_situ_acoustiq')
sys.path.append(r'C:\Users\martin.schoreisz\git\PPBE_BdxMet\src')

import os, re
import pandas as pd
import numpy as np
import altair as alt
from Import_stockage_donnees.Import_export_bdd import FichierMesureBruitCsv, FichierCsvEnquete

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 50)

# DONNEES BRUIT

> ## Lecture des fichiers ods

In [36]:
# exemple de lecture 
test = FichierMesureBruitCsv(r'C:\Users\martin.schoreisz\Documents\temp\BruitRessenti\mesures\CSV\sono1_10RuePierreRonsard\sono1_02-04-2022.csv')
# test.dfNiveauSpectre.head(10)

In [ ]:
# transfert vers Bdd par dossier
dossierSrc = r'C:\Users\martin.schoreisz\Documents\temp\BruitRessenti\mesures\CSV\sono4_26RueFrancoisVillon'
transfertFichierMesure2Bdd(dossierSrc)

# DONNEES RESSENTI

## Import des données brutes  
Il s'agit de lire le fichier, de le nettoyer puis de le rendre plus compréhensible.  
Comme le fichier mélange des données fournies par les riverains directement par LimeSurvey et d'autre issus du transfert depuis des questionnaires papiers, il faut également s'assurer de la présence d'un identifiant unique par riverains, en créant des adresses mail factice pour les réoponses issues de questionaire papier.

In [3]:
ressenti = FichierCsvEnquete(r'C:\Users\martin.schoreisz\Box\Cerema\3E-Prod_Env_Risq\E6-Nuisances\E61-Acoustique\60-Etudes_Locales\2022-BruitRessentiBdxMet\4-Travaux_en_cours\Enquete_LimeSurvey\resultats_LimeSurvey_Bruts\resultats_complets_enquete_avec_papier.csv')

## Analyse des données de participants et déclarations  
Il s'agit de fournir des indicateurs de répartition des différentes valeurs de contexte et de déclaration fournies par les participants. Attention, les formulaires papiers ne permettaient pas de remplir tout les informations de contexte

<div class="alert alert-block alert-warning"><b>Attention:</b> Il n'y a pas d'analyse des commentaires réalisée pour le moment.</div>

In [16]:
#creation des donnees
dicoCharts = ressenti.creationDfAnalyseRetour()
# creation des charts, avec ou sans export
ressenti.creationCharts(dicoCharts, export=False)

In [21]:
# exemple
dicoCharts['sensibilite']['data']
dicoCharts['sensibilite']['chart']

alt.LayerChart(...)

# DONNEES MÉTÉO
L'objectif est double : 
1. fournir des rose des vents selon une période de temps définie
1. fournir le statut de propagation acoustique (favorable / défavorable) selon l'angle et une période de temps définie

## Import des données brutes  
Les données sont réparties sur 2 fichiers dans le dossier suivant C:\Users\martin.schoreisz\Box\Cerema\3E-Prod_Env_Risq\E6-Nuisances\E61-Acoustique\60-Etudes_Locales\2022-BruitRessentiBdxMet\4-Travaux_en_cours\mesures\station_meteo\datas

In [23]:
dossier = r'C:\Users\martin.schoreisz\Box\Cerema\3E-Prod_Env_Risq\E6-Nuisances\E61-Acoustique\60-Etudes_Locales\2022-BruitRessentiBdxMet\4-Travaux_en_cours\mesures\station_meteo\datas'
files = ['Mesures_Ech24_21-03-2022_04-04-2022.csv', 'Mesures_Ech24_04-04-2022_19-04-2022.csv']

In [27]:
dfBrute = pd.read_csv(os.path.join(dossier, files[0]), skiprows=2, usecols=[0, 1, 2, 4, 5, 6, 9, 10, 11, 13, 14, 15],
                      names=['rowNum', 'dateHeure', 'vtsVentHaut', 'rayonnement', 'tempHaut', 'hygroHaut', 
                            'dirVentHaut', 'dirVentBas', 'vitVentBas', 'pluie', 'tempBas', 'hygroBas'],
                     parse_dates=[1], dayfirst=True)

In [26]:
dfBrute

,rowNum,dateHeure,vtsVentHaut,rayonnement,tempHaut,hygoHaut,dirVentHaut,dirVentBas,vitVentBas,pluie,tempBas,hygoBas
0,1,2022-03-21 13:21:21,2.77,674.4,19.032,54.5,88.4,355.2,1.51,NaN,19.936,55.9
1,2,2022-03-21 13:22:21,2.77,670.6,18.913,53.3,106.7,355.2,1.26,0.0,19.936,56.2
2,3,2022-03-21 13:23:21,3.27,669.4,18.866,53.4,85.6,85.6,1.51,0.0,19.770,55.1
3,4,2022-03-21 13:24:21,3.53,670.6,18.747,53.5,98.3,87.0,1.51,0.0,19.627,55.7
4,5,2022-03-21 13:25:21,1.76,671.9,18.961,54.0,103.9,99.7,0.76,0.0,19.841,56.9
...,...,...,...,...,...,...,...,...,...,...,...,...
20196,20197,2022-04-04 13:57:21,1.76,766.9,10.907,37.0,67.4,57.6,1.26,0.0,12.268,39.2
20197,20198,2022-04-04 13:58:21,1.01,768.1,10.858,37.6,53.3,39.3,0.50,0.0,12.025,40.0
20198,20199,2022-04-04 13:59:21,1.26,766.9,10.785,37.5,61.8,73.0,1.01,0.0,11.807,39.8
20199,20200,2022-04-04 14:00:21,1.51,756.9,10.663,36.9,33.7,30.9,1.26,0.0,11.734,37.6
